In [91]:
import requests
import json
import pandas as pd
import csv
import psycopg2

# Extraction Layer

In [92]:
url = "https://realty-mole-property-api.p.rapidapi.com/randomProperties"

querystring = {"limit":"100000"}

headers = {
	"x-rapidapi-key": "44f9cb2ad8msh6f3d4d207145829p1f2707jsn317690134bb4",
	"x-rapidapi-host": "realty-mole-property-api.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

#print(response.json())

data = response.json()

# Save the data to a file
filename = "PropertyRecords.json"
with open(filename, "w") as file:
    json.dump(data, file, indent=4)

In [93]:
# read into a dataframe

ProjectRecords_pd = pd.read_json('PropertyRecords.json')

In [85]:
#ProjectRecords_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 27 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   addressLine1      500 non-null    object 
 1   city              500 non-null    object 
 2   state             500 non-null    object 
 3   zipCode           500 non-null    int64  
 4   formattedAddress  500 non-null    object 
 5   bedrooms          333 non-null    float64
 6   squareFootage     385 non-null    float64
 7   yearBuilt         371 non-null    float64
 8   county            500 non-null    object 
 9   assessorID        363 non-null    object 
 10  legalDescription  352 non-null    object 
 11  subdivision       312 non-null    object 
 12  zoning            210 non-null    object 
 13  ownerOccupied     329 non-null    float64
 14  bathrooms         371 non-null    float64
 15  lotSize           355 non-null    float64
 16  propertyType      403 non-null    object 
 1

In [55]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_colwidth', None)

# Transformation Layer

In [101]:
#Coverting dictionary columns into string
def convert_dict_columns_to_string(ProjectRecords_pd: pd.DataFrame) -> pd.DataFrame:
    """
    Converts all dictionary-type columns in a DataFrame to string format.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: A DataFrame with dictionary columns converted to strings.
    """
    for col in ProjectRecords_pd.columns:
        # Check if the column contains dictionary entries
        if ProjectRecords_pd[col].apply(lambda x: isinstance(x, dict)).any():
            ProjectRecords_pd[col] = ProjectRecords_pd[col].apply(lambda x: str(x) if isinstance(x, dict) else x)
    return ProjectRecords_pd
ProjectRecords_pd = convert_dict_columns_to_string(ProjectRecords_pd)

In [87]:
# Replace Nan Values with appropraite defaults or remove row/columns as neccesary
ProjectRecords_pd.fillna({
    "assessorID": "Unknown",
    "legalDescription": "Not available",
    "squareFootage": 0,
    "subdivision": "Not available",
    "yearBuilt": 0,
    "bathrooms": 0,
    "lotSize": 0,
    "propertyType": "Unknown",
    "lastSalePrice":0,
    "lastSaleDate": "Not available",
    "features": "None",
    "taxAssessment": "Not available",
    "owner": "Unknown",
    "propertyTaxes": "Not available",
    "bedrooms": 0,
    "ownerOccupied": 0,
    "zoning": "Unknown",
    "addressLine2": "Not available",
    "formattedAddress": "Not Available",
    "county": "Not available"
    
}, inplace=True)

In [64]:
# Create the fact table
fact_columns = ["addressLine1", "city", "state", "zipCode", "formattedAddress", "squareFootage", "yearBuilt", "bathrooms",
               "bedrooms", "lotSize", "longitude", "latitude"]
fact_table = ProjectRecords_pd[fact_columns]


In [65]:
# Create location Dimension
location_dim = ProjectRecords_pd[["addressLine1", "city", "state", "zipCode","county","longitude", "latitude"]].drop_duplicates()
location_dim.index.name = "location_id"


In [73]:
# Create sale Dimension
sales_dim = ProjectRecords_pd[['lastSalePrice', "lastSaleDate"]].drop_duplicates().reset_index(drop=True)
sales_dim.index.name = "sales_id"

In [102]:
# Create Property Features Dimension
features_dim = ProjectRecords_pd[["features", "propertyType", "zoning"]].drop_duplicates().reset_index(drop=True)
features_dim.index.name = "features_id"

In [107]:
# Saving fact and dimensions table in csv format
fact_table.to_csv("property_fact.csv", index=False)
location_dim.to_csv("location_dimension.csv", index=True)
sales_dim.to_csv("sales_dimension.csv", index=True)
features_dim.to_csv("features_dimension.csv", index=True)